#  Naive Bayes klasifikator

**"A probabilistic classifier based on applying Bayes' theorem with strong (naive) independence assumptions between the features"**

# Pregled
* Generativni modeli
* Naive Bayes klasifikator
* Primer rada sa tekstom: twitter sentiment analysis


*Napomena: formule u nastavku mogu biti matematički gledano blago neprecizne, ali su takve u cilju lakšeg razumevanja suštine*

# Generativni modeli
* Klasifikatori se uobičajeno dele na dve glavne grupe: **diskriminativni** i **generativni**
* **Diskriminativni modeli** modeluju uslovnu verovatnoću $P(C|X=x)$, tj. verovatnoću da neki ulazni feature ili feature vektor pripada određenoj klasi
  * Predikcije iz ovog modela izvlačimo direktno
  * Primeri: logistička regresija, k-NN, SVM, neuralne mreže
* **Generativni modeli** modeluju združenu distribuciju $P(X, C)$
  * Predikcije iz ovog modela izvlačimo kao $P(C|X) = P(X, C) / P(X)$, ali kako je za dato $X$ verovatnoća $P(X)$ fiksna, važi $P(C|X) \sim P(X, C)$
  * Primeri: HMM, Naive Bayes, VAE, GAN

# Naive Bayes klasifikator
* Klasifikator koji koristi Bajesovu formulu
* **Bajesova formula - podsetnik**: $P(C|X) = \frac{P(X|C)P(C)}{P(X)} = \frac{P(X|C)P(C)}{\sum{P(X|C)P(C)}}$
  * $P(C)$ = *prior*
  * $P(X|C)$ = *likelihood*, $P(X)$ = *evidence*
  * $P(C|X)$ = *posterior*
* Kao što je već pomenuto $P(C|X) \sim P(X|C)P(C)$
  * Dakle: "verovatnoća da feature vektor X pripada klasi C je proporcionalna verovatnoći da bi neki primerak klase C imao feature vektor X, skalirano sa ukupnom verovatnoćom pojavljivanja klase C"
* $P(C)$ lako računamo kao broj pojavljivanja klase C u trening skupu, podeljeno sa veličinom trening skupa
  * $P(C) = \frac{|Elementi~trening~skupa~klase~C|}{|Ceo~trening~skup|}$
* Kako izračunati $P(X|C)$? 
  * Naive Bayes (*naivno*) pretpostavlja *nezavisnost ulaznih promenljivih* , pa je $P(X|C) = P(x_1|C) \cdot P(x_2|C) \cdot \dots \cdot P(x_N|C)$
* Ali kako izračunati $P(x_1|C)$, tj. verovatnoću da bi neki primerak klase $C$ imao prvi feature jednak $x_1$?
  * U obradi teksta se često koristi *Multinomial Naive Bayes* sa BoW reprezentacijom koja čuva brojeve pojavljivanja
    * Podsetnik: u BoW jedan feature je jedna reč tj. broj pojavljivanja te reči u tekstu
  * Sada je dakle $P(x_1|C)$ verovatnoća da neka klasa dokumenata sadrži reč $x_1$, što lako računamo kao frekvenciju ove reči u tekstovima za koje znamo da su iz klase $C$
  * $P(Reč_i|Klasa) = \frac{broj\_pojavljivanja(Reč_i, Klasa)}{ukupan\_broj\_reči(Klasa)}$
  * **Nazad na početak**: $P(Klasa|Skup Reči) \cdot P(Klasa) \times P(Reč_1 | Klasa) \times P(Reč_2 | Klasa) \cdot ... \cdot P(Reč_N | Klasa)$
  * **Predstavljeno preko BoW**: $P(Klasa|BoW vektor) \sim P(Klasa) \cdot \prod{P(Reč_i|Klasa)^{BoW[Reč_i]}}$
  * Očigledno, biramo klasu za koju je ova vrednost najveća
* **Problem 1**: kada je $N$ veliko množimo/stepenujemo mnogo brojeva od 0 do 1 što loše utiče na preciznost (vrlo lako možemo otići u nulu)
  * Rešenje: logaritmujemo sve! 
  * Sada je $P(Klasa|BoW~vektor) \sim \log(P(Klasa)) + \sum{BoW[Reč_i]\cdot\log(P(Reč_i|Klasa))}$
* **Problem 2**:  Šta ako je $broj\_pojavljivanja(Reč_i, Klasa)=0$? Tada $P(Reč_i|Klasa)$ postaje 0 paceo proizvod postaje 0
  * Rešenje: **Laplace Smoothing**
  * Uvodimo konstantu $\alpha$ (uglavnom jednaku 1) i smatramo da svaka **klasa** sadrži barem toliko od svake reči
  * Dakle, na nivou svake **klase** (ne dokumenta!) dodajemo $\alpha$ puta svaku reč iz vokabulara
 * Nova formula: $P(Reč_i|Klasa) = \frac{broj\_pojavljivanja(Reč_i, Klasa) + \alpha}{ukupan\_broj\_reči(Klasa) + |Vocab| \cdot \alpha}$
* [Više o Naive Bayes](http://cs229.stanford.edu/notes/cs229-notes2.pdf)

## Naive Bayes - konačne formule
* $P(Klasa|BoW vektor) \sim P(Klasa) \cdot \prod{P(Reč_i|Klasa)^{BoW[Reč_i]}}$


---


* $P(Klasa) = \frac{|Elementi~trening~skupa~klase~Klasa|}{|Ceo~ trening~skup|}$


---


* $P(Reč_i|Klasa) = \frac{broj\_pojavljivanja(Reč_i, Klasa) + \alpha}{ukupan\_broj\_reči(Klasa) + |Vocab| \cdot \alpha}$

In [4]:
import numpy as np

class MultinomialNaiveBayes:
  def __init__(self, nb_classes, nb_words, pseudocount):
    self.nb_classes = nb_classes
    self.nb_words = nb_words
    self.pseudocount = pseudocount
  
  def fit(self, X, Y):
    nb_examples = X.shape[0]

    # Racunamo P(Klasa) - priors
    # np.bincount nam za datu listu vraca broj pojavljivanja svakog celog
    # broja u intervalu [0, maksimalni broj u listi]
    self.priors = np.bincount(Y) / nb_examples
    print('Priors:')
    print(self.priors)

    # Racunamo broj pojavljivanja svake reci u svakoj klasi
    occs = np.zeros((self.nb_classes, self.nb_words))
    for i in range(nb_examples):
      c = Y[i]
      for w in range(self.nb_words):
        cnt = X[i][w]
        occs[c][w] += cnt
    print('Occurences:')
    print(occs)
    
    # Racunamo P(Rec_i|Klasa) - likelihoods
    self.like = np.zeros((self.nb_classes, self.nb_words))
    for c in range(self.nb_classes):
      for w in range(self.nb_words):
        up = occs[c][w] + self.pseudocount
        down = np.sum(occs[c]) + self.nb_words*self.pseudocount
        self.like[c][w] = up / down
    print('Likelihoods:')
    print(self.like)
          
  def predict(self, bow):
    # Racunamo P(Klasa|bow) za svaku klasu
    probs = np.zeros(self.nb_classes)
    for c in range(self.nb_classes):
      prob = np.log(self.priors[c])
      for w in range(self.nb_words):
        cnt = bow[w]
        prob += cnt * np.log(self.like[c][w])
      probs[c] = prob
    # Trazimo klasu sa najvecom verovatnocom
    print('\"Probabilites\" for a test BoW (with log):')
    print(probs)
    prediction = np.argmax(probs)
    return prediction
  
  def predict_multiply(self, bow):
    # Racunamo P(Klasa|bow) za svaku klasu
    # Mnozimo i stepenujemo kako bismo uporedili rezultate sa slajdovima
    probs = np.zeros(self.nb_classes)
    for c in range(self.nb_classes):
      prob = self.priors[c]
      for w in range(self.nb_words):
        cnt = bow[w]
        prob *= self.like[c][w] ** cnt
      probs[c] = prob
    # Trazimo klasu sa najvecom verovatnocom
    print('\"Probabilities\" for a test BoW (without log):')
    print(probs)
    prediction = np.argmax(probs)
    return prediction

# Primer: https://web.stanford.edu/class/cs124/lec/naivebayes.pdf - slajd 44

# Klase: (china, japan)
# Vocab: (Chinese, Beijing, Shanghai, Macao, Tokyo, Japan)
class_names = ['China', 'Japan']
Y = np.asarray([0, 0, 0, 1])
X = np.asarray([
  [2, 1, 0, 0, 0, 0],
  [2, 0, 1, 0, 0, 0],
  [1, 0, 0, 1, 0, 0],
  [1, 0, 0, 0, 1, 1]
])
test_bow = np.asarray([3, 0, 0, 0, 1, 1])

model = MultinomialNaiveBayes(nb_classes=2, nb_words=6, pseudocount=1)
model.fit(X, Y)
prediction = model.predict(test_bow)
print('Predicted class (with log): ', class_names[prediction])
prediction = model.predict_multiply(test_bow)
print('Predicted class (without log): ', class_names[prediction])


Priors:
[0.75 0.25]
Occurences:
[[5. 1. 1. 1. 0. 0.]
 [1. 0. 0. 0. 1. 1.]]
Likelihoods:
[[0.42857143 0.14285714 0.14285714 0.14285714 0.07142857 0.07142857]
 [0.22222222 0.11111111 0.11111111 0.11111111 0.22222222 0.22222222]]
"Probabilites" for a test BoW (with log):
[-8.10769031 -8.90668135]
Predicted class (with log):  China
"Probabilities" for a test BoW (without log):
[0.00030121 0.00013548]
Predicted class (without log):  China


# Primer rada sa tekstom: twitter sentiment analysis
*  **Problem**: klasifikovati tvitove na osnovu "sentimenta": (neutralan, pozitivan, negativan)
*  [Kaggle dataset](https://www.kaggle.com/c/twitter-sentiment-analysis2)
*  Potreban ceo proces čišćenja i obrade podataka
*  Potrebna feature-izacija (npr. BoW)
*  Jedan od mogućih modela može biti Multinomial Naive Bayes
*  Potrebna podela skupa podataka na trening/validacioni/test skup radi evaluacije rešenja
*  [Primer rešenja](https://github.com/rand0musername/twitter-sentiment)

